In [127]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
from algo_pro import *
from evaluation import *
from jqdatasdk import *

auth('13692899351', 'M0a6n1d7y')
class Context():
    def __init__(self,start,end,securities,amt):

        self.start = datetime.strptime(start,'%Y-%m-%d')
        self.end = datetime.strptime(end,'%Y-%m-%d')
        self.trade_days = list(get_trade_days(start,end))        
        self.secty = securities
        self.position = {key:0 for key in securities}
        self.current_dt = self.trade_days[0]
        self.previous_date = self.current_dt-timedelta(days=1)
        self.his_posi = pd.DataFrame([],columns = securities)
        self.total_amt = amt


    def next_day(self):
        pos = pd.DataFrame(self.position.values()).T
        pos.columns=self.secty
        self.his_posi = pd.concat([self.his_posi, pos])
        self.previous_date = self.current_dt
        self.current_dt = self.trade_days[self.trade_days.index(self.current_dt)+1]

    def chg_position(self,posi_dict):
        if len(posi_dict):
            pre_prs = get_price(list(posi_dict.keys()),self.current_dt,self.current_dt,fields=['open'])

            for k,v in posi_dict.items():
                val = pre_prs[pre_prs['code']==k]
                self.position[k] = v / val.iloc[0,2]

        prs = get_price( self.secty, self.trade_days[self.trade_days.index(self.current_dt)+1], self.trade_days[self.trade_days.index(self.current_dt)+1], fields=['open'] )
        s = (np.array(list(self.position.values())) * prs['open'].values).sum()
        self.total_amt = s if s else self.total_amt


    @property
    def history_posi(self):
        self.his_posi.index = get_trade_days(self.start,self.end+timedelta(days=1))
        return self.his_posi

class Backtest():
    def __init__(self, start, end, security, total_amt):
        # print(1)
        self.start = start
        self.end = end
        self.security = security
        self.stocklist = get_index_stocks(security)
        self.context = Context(start,end,self.stocklist,total_amt)
        self.total_amt = total_amt


    def backtest(self):
        trade_days = get_trade_days(self.start,self.end)

        initialize(self.context)
        for day in trade_days:

            before_market_open(self.context)
            val_posi = rebalance(self.context)
            # if len(val_posi):
            self.context.chg_position(val_posi)
            self.context.next_day()

        self.history_posi = self.context.history_posi
        # print(self.history_posi)

        data = get_price(self.stocklist, start_date = self.start,end_date = self.end)
        data = data.loc[:,['time','code','close']].pivot(index = 'time',columns = 'code')#.droplevel(0, axis=1)
        data = data.droplevel(0, axis=1)
        performance = EvaluationHandler(self.history_posi, data,self.total_amt)
        self.shrp = performance.sharpe
        self.mdd = performance.maxdrawdown
        self.profit = performance.profit
        self.pnl = performance.pnl

In [128]:

bt = Backtest('2017-05-15','2021-02-03','000300.XSHG',100000)
result = bt.backtest()
print(bt.shrp)
print(bt.mdd)
# print()

c:\ProgramData\Anaconda3\lib\site-packages\jqdatasdk\api.py:34: PanelObsoleteWarning: 当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口的 panel 参数将固定为 False（0.25 及以上版本的 pandas 不再支持 panel，如使用该数据结构和相关函数请注意修改）
  warnings.warn(PandasChecker.VERSION_NOTICE_MESSAGE, PanelObsoleteWarning)


2017-05-14
北上资金均值：2.89  北上资金上界：27.53 北上资金下界：-11.66
2017-05-15
北上资金均值：17.57  北上资金上界：26.83 北上资金下界：-10.02
2017-05-16
北上资金均值：31.62  北上资金上界：27.75 北上资金下界：-10.28


c:\ProgramData\Anaconda3\lib\site-packages\jqdatasdk\api.py:34: PanelObsoleteWarning: 当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口的 panel 参数将固定为 False（0.25 及以上版本的 pandas 不再支持 panel，如使用该数据结构和相关函数请注意修改）
  warnings.warn(PandasChecker.VERSION_NOTICE_MESSAGE, PanelObsoleteWarning)


{'601888.XSHG', '000858.XSHE', '002415.XSHE', '603288.XSHG', '600519.XSHG', '600585.XSHG', '600276.XSHG', '601939.XSHG', '600690.XSHG', '601318.XSHG', '000651.XSHE', '600031.XSHG', '601021.XSHG', '600900.XSHG', '000333.XSHE', '600741.XSHG', '600887.XSHG'}
2017-05-17
北上资金均值：15.55  北上资金上界：27.81 北上资金下界：-9.90
2017-05-18
北上资金均值：10.11  北上资金上界：27.83 北上资金下界：-9.69
2017-05-21
北上资金均值：13.71  北上资金上界：27.84 北上资金下界：-9.31
2017-05-22
北上资金均值：6.45  北上资金上界：27.82 北上资金下界：-9.15
2017-05-23
北上资金均值：3.90  北上资金上界：27.80 北上资金下界：-9.07
2017-05-24
北上资金均值：11.96  北上资金上界：27.84 北上资金下界：-8.84
2017-05-25
北上资金均值：16.74  北上资金上界：27.82 北上资金下界：-8.29
2017-05-30
北上资金均值：0.44  北上资金上界：27.76 北上资金下界：-8.17
2017-05-31
北上资金均值：-15.45  北上资金上界：27.81 北上资金下界：-8.23


c:\ProgramData\Anaconda3\lib\site-packages\jqdatasdk\api.py:34: PanelObsoleteWarning: 当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口的 panel 参数将固定为 False（0.25 及以上版本的 pandas 不再支持 panel，如使用该数据结构和相关函数请注意修改）
  warnings.warn(PandasChecker.VERSION_NOTICE_MESSAGE, PanelObsoleteWarning)


...

c:\ProgramData\Anaconda3\lib\site-packages\jqdatasdk\api.py:34: PanelObsoleteWarning: 当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口的 panel 参数将固定为 False（0.25 及以上版本的 pandas 不再支持 panel，如使用该数据结构和相关函数请注意修改）
  warnings.warn(PandasChecker.VERSION_NOTICE_MESSAGE, PanelObsoleteWarning)


{'601318.XSHG'}
2018-11-05
北上资金均值：11.88  北上资金上界：73.45 北上资金下界：-47.88
2018-11-06
北上资金均值：-15.00  北上资金上界：73.14 北上资金下界：-48.55
2018-11-07
北上资金均值：22.93  北上资金上界：73.02 北上资金下界：-48.56
2018-11-08
北上资金均值：40.91  北上资金上界：73.60 北上资金下界：-47.79
2018-11-11
北上资金均值：3.84  北上资金上界：73.57 北上资金下界：-47.88
2018-11-12
北上资金均值：14.32  北上资金上界：73.65 北上资金下界：-47.46
2018-11-13
北上资金均值：26.74  北上资金上界：73.95 北上资金下界：-47.21
2018-11-14
北上资金均值：-6.53  北上资金上界：73.49 北上资金下界：-46.08
2018-11-15
北上资金均值：28.16  北上资金上界：73.53 北上资金下界：-46.07
2018-11-18
北上资金均值：22.69  北上资金上界：73.69 北上资金下界：-45.48
2018-11-19
北上资金均值：19.14  北上资金上界：73.80 北上资金下界：-45.37
2018-11-20
北上资金均值：-28.15  北上资金上界：74.07 北上资金下界：-46.39
2018-11-21
北上资金均值：12.78  北上资金上界：74.08 北上资金下界：-45.83
2018-11-22
北上资金均值：-3.76  北上资金上界：73.86 北上资金下界：-46.22
2018-11-25
北上资金均值：-7.44  北上资金上界：73.05 北上资金下界：-46.58
2018-11-26
北上资金均值：26.73  北上资金上界：72.81 北上资金下界：-46.56
2018-11-27
北上资金均值：4.21  北上资金上界：72.79 北上资金下界：-46.62
2018-11-28
北上资金均值：28.76  北上资金上界：73.13 北上资金下界：-46.30
2018-11-29
北上资金均值：12.88  北上资金上界：73.04 北上资金下界：-46

c:\ProgramData\Anaconda3\lib\site-packages\jqdatasdk\api.py:34: PanelObsoleteWarning: 当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口的 panel 参数将固定为 False（0.25 及以上版本的 pandas 不再支持 panel，如使用该数据结构和相关函数请注意修改）
  warnings.warn(PandasChecker.VERSION_NOTICE_MESSAGE, PanelObsoleteWarning)


2018-12-04
北上资金均值：45.63  北上资金上界：80.86 北上资金下界：-49.23
2018-12-05
北上资金均值：11.38  北上资金上界：80.91 北上资金下界：-49.03
2018-12-06
北上资金均值：-39.97  北上资金上界：81.18 北上资金下界：-50.78
2018-12-09
北上资金均值：-2.44  北上资金上界：80.98 北上资金下界：-51.15
2018-12-10
北上资金均值：-15.52  北上资金上界：81.04 北上资金下界：-51.57
2018-12-11
北上资金均值：5.26  北上资金上界：80.76 北上资金下界：-51.79
2018-12-12
北上资金均值：14.74  北上资金上界：80.72 北上资金下界：-51.82
2018-12-13
北上资金均值：68.91  北上资金上界：82.38 北上资金下界：-52.08
2018-12-16
北上资金均值：-9.50  北上资金上界：82.12 北上资金下界：-52.63
2018-12-17
北上资金均值：15.31  北上资金上界：81.95 北上资金下界：-52.71
2018-12-18
北上资金均值：-27.79  北上资金上界：81.75 北上资金下界：-53.86
2018-12-19
北上资金均值：-5.22  北上资金上界：81.69 北上资金下界：-54.16
2018-12-20


Exception: 您当天的查询条数超过了每日最大查询限制：50万条；付费可增加流量权限，详情请咨询管理员，微信号：15815705123

In [124]:
s = [1,2,5,6,3,7]
s.index(4)


ValueError: 4 is not in list

In [81]:
prs[prs['code']=='000300.XSHG'].loc[0,'open']

3313.95